# Real neurons with total path length larger than 40 micron

In [ ]:
# this module keeps the code update, so I can use new function directly after modifying code
using PyPlot
using Revise

using RealNeuralNetworks.Neurons
using JLD2
using RealNeuralNetworks.SWCs


const DATASET_DIR = expanduser("../01_data")

NEURON_DIR = "$(DATASET_DIR)/atlas/bin"

using RealNeuralNetworks
using RealNeuralNetworks.SWCs
using RealNeuralNetworks.Neurons
using RealNeuralNetworks.Neurons.Segments
using DataFrames
using Plots
using Query
using JLD2
using Statistics
using LinearAlgebra
using JuliaDB
using ProgressMeter

# Plots.plotly()
# Plots.InspectDRBackend()
# Plots.pyplot()

df = DataFrame()

dirName = NEURON_DIR
@show dirName

In [74]:
using JSON
DATASET_DIR

neuronIdList = JSON.parsefile(joinpath(DATASET_DIR, "neuron_ids.json"))["valid"]

# using DataStructures
neuronId2neuron = Dict{Int,Neuron}()

skippedNeuronIdList = Int[]
newNeuronIdList = Int[]
totalPathLengthList = Float32[]

# @showprogress 1 "read neurons" for (index, fileName) in enumerate( readdir( NEURON_DIR  ))
@showprogress 1 "read neurons" for neuronId in neuronIdList
#     fullName = joinpath(NEURON_DIR, fileName)
#     neuronId = Meta.parse(split(fileName, ".")[1])
    fullName = joinpath(DATASET_DIR, "atlas/bin/$(neuronId).bin")
    neuron =  Neurons.load(fullName)
    
    totalPathLength = Neurons.get_total_path_length( neuron )  
    
    
#     neuronId2neuron[neuronId] = neuron
    # only keep the neurons with path length larger than 40 microns
    if Neurons.get_total_path_length(neuron) > Float32(20000)
        push!(totalPathLengthList, totalPathLength)
        push!(newNeuronIdList, neuronId)
        neuronId2neuron[neuronId] = neuron
    else
        push!(skippedNeuronIdList, neuronId)
    end
end  

neuronIdList = newNeuronIdList
df[!,:neuronId] = neuronIdList;


println("skipped ", length(skippedNeuronIdList), " neurons: ", skippedNeuronIdList)

LoadError: UndefVarError: DATASET_DIR not defined

In [38]:
println("skipped ", length(skippedNeuronIdList), " neurons: ", skippedNeuronIdList)
d = JSON.parsefile(joinpath(DATASET_DIR, "neuron_ids.json"))
d["analysis"] = neuronIdList
open(joinpath(DATASET_DIR, "neuron_ids.json"), "w") do f
    write(f, JSON.json(d))
end

LoadError: UndefVarError: skippedNeuronIdList not defined

In [39]:

df[!, :totalPathLength] = totalPathLengthList

p = PyPlot.hist(df[!, :totalPathLength]./1000, bins=80)
PyPlot.xlabel("total path length (\u03BCm)")
PyPlot.ylabel("count of cells")

# println("total path length is ", sum( totalPathLengthList ) / 1e6, " mm")

fname = expanduser("~/Pictures/total_path_length.svg")

PyPlot.savefig(fname)

LoadError: UndefVarError: df not defined

In [40]:

# PyPlot.plt[:hist](totalPathLengthList./1000, bins=40)
# PyPlot.xlabel(L"total\ path\ length\ (\mu m)")
# PyPlot.ylabel("number of neurons")

# p = PyPlot.hist(totalPathLengthList./1000, leg=false, nbins=80)
# xlabel!(p, L"total\ path\ length\ (\mu m)")
# ylabel!(p, "number of neurons")

# PyPlot.xlim(0,100)
# PyPlot.savefig("~/Pictures/total_path_length.svg")
# PyPlot.savefig(expanduser("~/Pictures/total_path_length.png"))

In [41]:
# fname = expanduser("~/Pictures/total_path_length.png")

# PyPlot.savefig(fname)
# PyPlot.SVG(fname)
# PyPlot.plt[:savefig](fname)


In [42]:
# ?PyPlot.savefig
PyPlot.PyCall.python

LoadError: UndefVarError: PyPlot not defined

In [43]:
totalAxonPathLengthList = Vector{Float32}()
for (neuronId, neuron) in neuronId2neuron 
    #println("compute total path length of $cellId ...")
    totalAxonPathLength = Neurons.get_total_path_length( neuron; class=Neurons.Segments.AXON_CLASS )  
    push!(totalAxonPathLengthList, totalAxonPathLength)
end  

df[!, :totalAxonPathLength] = totalAxonPathLengthList

p = PyPlot.hist(df[!, :totalAxonPathLength]./1000,  bins=50)
PyPlot.xlabel(L"total\ axon\ path\ length\ (\mu m)")
PyPlot.ylabel(L"count\ of\ cells")
PyPlot.savefig(expanduser("~/Pictures/total_axon_path_length.svg"))

LoadError: UndefVarError: neuronId2neuron not defined

In [44]:
totalDendPathLengthList = Vector{Float32}()
for (neuronId, neuron) in neuronId2neuron 
    #println("compute total path length of $cellId ...")
    totalDendPathLength = Neurons.get_total_path_length( neuron; class=Neurons.Segments.DENDRITE_CLASS )  
    push!(totalDendPathLengthList, totalDendPathLength)
end  

df[!, :totalDendPathLength] = totalDendPathLengthList

p = hist(df[!, :totalDendPathLength]./1000, bins=50)
PyPlot.xlabel(L"total\ dendrite\ path\ length\ (\mu m)")
PyPlot.ylabel(L"count\ of\ cells")
xlim(0, 600)
PyPlot.savefig(expanduser("~/Pictures/total_dendrite_path_length.svg"))

LoadError: UndefVarError: neuronId2neuron not defined

In [45]:
preSynapseNumList = Vector{Int}()
postSynapseNumList = Vector{Int}()
for (neuronId, neuron) in neuronId2neuron 
    #println("compute total path length of $cellId ...")
    preSynapseNum = Neurons.get_num_pre_synapses( neuron )  
    push!(preSynapseNumList, preSynapseNum)
    
    postSynapseNum = Neurons.get_num_post_synapses( neuron )  
    push!(postSynapseNumList, postSynapseNum)
end  

df[!, :preSynapseNum] = preSynapseNumList
df[!, :postSynapseNum] = postSynapseNumList;

p = hist(df[!, :preSynapseNum]./1000, bins=50)
# PyPlot.scatter()
xlabel(L"presynapse number")
ylabel(L"count\ of\ cells")
xlim(0, 0.5)

PyPlot.savefig(expanduser("~/Pictures/presynapse_num.svg"))

LoadError: UndefVarError: neuronId2neuron not defined

In [46]:


p = hist(df[!, :postSynapseNum]./1000, bins=50)
# PyPlot.scatter()
xlabel(L"postsynapse number")
ylabel(L"count\ of\ cells")
xlim(0, 0.8)

PyPlot.savefig(expanduser("~/Pictures/postsynapse_num.svg"))

LoadError: UndefVarError: df not defined

In [47]:
const ROOT_RADIUS_THRESHOLD = 800
rootRadiusList = Vector{Float32}()
@showprogress 1 "compute root radius..." for (neuronId,neuron) in neuronId2neuron
    #root = Neurons.get_root_node(neuron)
    root = Neurons.get_root_node( neuron )
    push!(rootRadiusList, root[4])
end 

df[!, :rootRadius] = rootRadiusList

p = PyPlot.hist( rootRadiusList./1000, bins = 70 )
xlabel(L"root\ node\ radius\ (\mu m)")
ylabel(L"count\ of\ neurons")

# PyPlot.plt[:hist](rootRadiusList./1000, bins=70)
# PyPlot.xlabel(L"root\ node\ radius\ (\mu m)")
# PyPlot.ylabel("count of neurons")

PyPlot.savefig(expanduser("~/Pictures/root_radius.svg"))

LoadError: LoadError: UndefVarError: @showprogress not defined
in expression starting at In[47]:3

In [48]:
# using Query
# q = @from i in df begin
#      @where i.rootRadius > ROOT_RADIUS_THRESHOLD
#      @select {i.neuronId}
#      @collect DataFrame
# end
# neuronIdListWithSoma = q[:neuronId]
# @save "neuron_id_list_with_soma.jld" neuronIdListWithSoma

In [49]:
averageBranchingAngleList = Vector{Float32}()
for (neuronId, neuron) in neuronId2neuron
    branchingAngleList = map(i->Neurons.get_branching_angle(neuron,i), 
                                                        1:Neurons.get_num_segments(neuron))
    push!(averageBranchingAngleList, mean(branchingAngleList))
end 

df[!, :averageBranchingAngle] = averageBranchingAngleList

# PyPlot.plt[:hist](averageBranchingAngleList, bins=90)
# PyPlot.xlabel("average branching angle")
# PyPlot.ylabel("count of neurons")

PyPlot.hist(averageBranchingAngleList, bins=50)
xlabel("average branching angle")
ylabel("count of neurons")


PyPlot.savefig(expanduser("~/Pictures/average_branching_angle.svg"))

LoadError: UndefVarError: neuronId2neuron not defined

In [50]:
using RealNeuralNetworks.Neurons.Segments
averageTortuosityList = Vector{Float32}()
for (neuronId, neuron) in neuronId2neuron
    tortuosityList = map(Neurons.Segments.get_tortuosity, Neurons.get_segment_list(neuron))
    push!(averageTortuosityList, mean(tortuosityList))
end 

df[!, :averageTortuosity] = averageTortuosityList

# PyPlot.plt[:hist](averageTortuosityList, bins=20)
# PyPlot.xlabel("average tortuosity")
# PyPlot.ylabel("count of neurons")

hist(averageTortuosityList, bins=64)
xlabel("average tortuosity")
ylabel("count of neurons")
xlim(1, 1.6)


PyPlot.savefig(expanduser("~/Pictures/average_tortuosity.svg"))

LoadError: ArgumentError: Package RealNeuralNetworks not found in current path:
- Run `import Pkg; Pkg.add("RealNeuralNetworks")` to install the RealNeuralNetworks package.


In [51]:
asymmetryList = map(Neurons.get_asymmetry, values(neuronId2neuron))
df[!, :asymmetry] = asymmetryList



hist(df[!, :asymmetry]./1000, bins=40)
title("Asymmetry")

xlabel(L"euclidean\ distance\ from\ soma\ to\ center\ of\ skeleton\ mass\ (\mu m)")
ylabel(L"count\ of\ neurons")


PyPlot.savefig(expanduser("~/Pictures/asymmetry.svg"))

LoadError: UndefVarError: Neurons not defined

In [52]:
typicalRadiusList = map(Neurons.get_typical_radius, values(neuronId2neuron))
df[!, :typicalRadius] = typicalRadiusList

PyPlot.hist(df[!, :typicalRadius]./1000, bins=40)
xlabel(L"typical\ radius\ (\mu m)")
ylabel(L"count\ of\ neurons")


PyPlot.savefig(expanduser("~/Pictures/typical_radius.svg"))

LoadError: UndefVarError: Neurons not defined

In [53]:
numBranchingPointList = Vector{Int}()
for (neuronId, neuron) in neuronId2neuron
    numBranchingPoint = Neurons.get_num_branching_points(neuron)    
    push!(numBranchingPointList, numBranchingPoint) 
end  

df[!, :numBranchingPoint] = numBranchingPointList

p = hist(df[!, :numBranchingPoint], bins=64)
xlabel("number of branching points")
ylabel("count of neurons")
xlim(0, 100)


PyPlot.savefig(expanduser("~/Pictures/branching_point_num.svg"))

LoadError: UndefVarError: neuronId2neuron not defined

In [54]:
# using Query
# q = @from i in df begin
#      @where i.rootRadius > ROOT_RADIUS_THRESHOLD
#      @select {numBranchingPoint=i.numBranchingPoint}
#      @collect DataFrame
# end
# p = histogram(q[:numBranchingPoint], leg=false, nbins=40)
# xlabel!(p, "number of branching points")
# ylabel!(p, "count of neurons")

In [55]:
# q = @from i in df begin
#      @where i.numBranchingPoint > 300
#      @select {neuronId = i.neuronId, numBranchingPoint=i.numBranchingPoint}
#      @collect DataFrame
# end
# q

In [56]:
medianBranchPathLengthList = Vector{Float64}()
maxBranchPathLengthList = Vector{Float64}()
for (neuronId, neuron) in neuronId2neuron
    branchPathLengthList = Neurons.get_segment_path_length_list( neuron )
    push!(medianBranchPathLengthList, median(branchPathLengthList))
    push!(maxBranchPathLengthList, maximum(branchPathLengthList))
end  

df[!, :medianBranchPathLength] = medianBranchPathLengthList
df[!, :maxBranchPathLength] = maxBranchPathLengthList 
p = hist(df[!, :medianBranchPathLength]./1000, bins=128 )
xlabel(L"median\ of\ branch\ path\ length\ (\mu m)")
ylabel(L"count\ of\ neurons")
xlim(0, 50)



PyPlot.savefig(expanduser("~/Pictures/median_of_branch_path_length.svg"))

LoadError: UndefVarError: neuronId2neuron not defined

In [57]:
using Query
q = @from i in df begin
     @where i.rootRadius > ROOT_RADIUS_THRESHOLD
     @select {medianBranchPathLength=i.medianBranchPathLength}
     @collect DataFrame
end
p = hist(q[!, :medianBranchPathLength]./1000, bins=40)
xlabel(L"median\ of\ branch\ path\ length\ with\ soma\ (\mu m)")
ylabel(L"count\ of\ neurons")


PyPlot.savefig(expanduser("~/Pictures/median_of_branch_path_length_with_soma.svg"))

LoadError: ArgumentError: Package Query not found in current path:
- Run `import Pkg; Pkg.add("Query")` to install the Query package.


In [58]:
p = hist(df[!, :maxBranchPathLength]./1000, bins=64)
xlabel(L"maximum\ branch\ path\ length\ (\mu m)")
ylabel(L"count\ of\ neurons")


PyPlot.savefig(expanduser("~/Pictures/maximum_branch_path_length.svg"))

LoadError: UndefVarError: df not defined

In [59]:
using Query
q = @from i in df begin
     @where i.rootRadius > ROOT_RADIUS_THRESHOLD
     @select {maxBranchPathLength=i.maxBranchPathLength}
     @collect DataFrame
end
p = hist(q[!, :maxBranchPathLength]./1000, bins=64)
xlabel(L"maximum\ branch\ path\ length\ with\ soma\ (\mu m)")
ylabel(L"count\ of\ neurons")
xlim(0, 175)

PyPlot.savefig(expanduser("~/Pictures/maximum_branch_path_length_with_soma.svg"))

LoadError: ArgumentError: Package Query not found in current path:
- Run `import Pkg; Pkg.add("Query")` to install the Query package.


### Convex Hull

In [60]:
# # remove cell 76388 which is too small to compute convex hull
# q = @from i in df begin
#      @where i.neuronId != 76388 && i.neuronId!=79514
#      @select i
#      @collect DataFrame
# end
# df = q

In [61]:
# convex hull analysis
using QHull
using ProgressMeter

convexHullVolumeList = Float32[]
convexHullAreaList = Float32[]
@showprogress 1 "computing convex hull..." for neuronId in keys(neuronId2neuron)
    neuron = neuronId2neuron[neuronId]
    nodeList = Neurons.get_node_list(neuron)
    points = zeros((length(nodeList), 3))
    for (i,node) in enumerate(nodeList)
        points[i,:] = [nodeList[i][1:3]...]
    end 
    ch = chull(points)
    push!(convexHullVolumeList, ch.volume)
    push!(convexHullAreaList, ch.area)
end

df[!, :convexHullVolume] = Vector{Float32}( convexHullVolumeList )
df[!, :convexHullArea] = Vector{Float32}( convexHullAreaList )

p = hist(convexHullVolumeList ./ 1e9, bins=64)
xlabel(L"convex\ hull\ volume\ (\mu m^3)")
ylabel(L"count\ of\ neurons")


PyPlot.savefig(expanduser("~/Pictures/convex_hull_volume.svg"))

LoadError: ArgumentError: Package QHull not found in current path:
- Run `import Pkg; Pkg.add("QHull")` to install the QHull package.


In [62]:
# frustum-based surface area and volume
surfaceAreaList = Float32[]
volumeList = Float32[]

@showprogress 1 "compute surface area..." for neuronId in keys(neuronId2neuron)
    neuron = neuronId2neuron[neuronId]
    push!(surfaceAreaList, Neurons.get_surface_area(neuron))
    push!(volumeList, Neurons.get_volume(neuron))
end

df[!, :surfaceArea] = surfaceAreaList
df[!, :volume] = volumeList

p = hist(surfaceAreaList./1e6, bins=40)
xlabel(L"surface\ area\ (\mu m^2)")
ylabel(L"count\ of\ neurons")


PyPlot.savefig(expanduser("~/Pictures/surface_area.svg"))

LoadError: LoadError: UndefVarError: @showprogress not defined
in expression starting at In[62]:5

In [63]:
p = hist(volumeList./1e9, bins=64)
xlabel(L"volume\ (\mu m^3)")
ylabel(L"count\ of\ neurons")
xlim(0, 150)


PyPlot.savefig(expanduser("~/Pictures/volume.svg"))

LoadError: UndefVarError: hist not defined

# Save features

In [64]:
# save dataframe
using CSV
@show size(df)
@assert DataFrames.ncol(df) >= 14
CSV.write("$(DATASET_DIR)/features.csv", df)

LoadError: ArgumentError: Package CSV not found in current path:
- Run `import Pkg; Pkg.add("CSV")` to install the CSV package.


# Clustering

In [65]:
# data frame only contain neurons
# using Que
ndf = @from i in df begin
     @where i.rootRadius > ROOT_RADIUS_THRESHOLD
     @select i
     @collect DataFrame
end

# ndf[:convexHullVolume] |> Vector{Float64}
@show nrow(ndf)

LoadError: LoadError: UndefVarError: @from not defined
in expression starting at In[65]:3

In [66]:
names(ndf)


LoadError: UndefVarError: ndf not defined

In [67]:
using StatsBase
# each column is a sample
sampleMatrix = zeros( size(ndf,2), size(ndf,1)) 
@show size(sampleMatrix)

nameIndex = 0
for name in names(ndf)
    if name != "neuronId"
        nameIndex += 1
#         @show name, typeof(ndf[name]), nameIndex
        sampleMatrix[ nameIndex, : ] = zscore(ndf[!, name])
    end 
end 

# construct distance matrix
using Distances
distanceMatrix = pairwise( CosineDist(), sampleMatrix )
# @show size(distanceMatrix)
# @show maximum(distanceMatrix)
# @show minimum(distanceMatrix)
# @show cosine_dist(sampleMatrix[1,:], sampleMatrix[2,:])
# @show sampleMatrix
# hierachical clustering
using Clustering
clust = Clustering.hclust( distanceMatrix, :average);


LoadError: ArgumentError: Package StatsBase not found in current path:
- Run `import Pkg; Pkg.add("StatsBase")` to install the StatsBase package.


In [68]:
plotly()
import Pkg
include(joinpath(Pkg.dir("RealNeuralNetworks"), "src/Utils/PlotRecipes.jl")); using .PlotRecipes

PlotRecipes.plot(clust)

LoadError: UndefVarError: plotly not defined

In [69]:
@show size(sampleMatrix)
@show maximum(distanceMatrix)
@show minimum(distanceMatrix)
@show cosine_dist(sampleMatrix[:,110], sampleMatrix[:, 13])
x = sampleMatrix[:, 110]
y = sampleMatrix[:, 13]
# @show dot(x, y) / (norm(x) * norm(y))
for i in 1:size(distanceMatrix,1)
    for j in 1:size(distanceMatrix, 2)
        if distanceMatrix[i,j] > 1.9
            print("$i,$j:$(distanceMatrix[i,j]); ")
        end
    end 
end


LoadError: UndefVarError: sampleMatrix not defined

In [70]:
#plot heat map
# gr()
# rearrange distance matrix according to order
using RealNeuralNetworks.NeuralNets
reorderedDistanceMatrix = NeuralNets.reorder_matrix(distanceMatrix, clust.order)

PyPlot.imshow(reorderedDistanceMatrix)
title("distance matrix based on morphological statistics")
colorbar()



PyPlot.savefig(expanduser("~/Pictures/distance_matrix.png"))

LoadError: ArgumentError: Package RealNeuralNetworks not found in current path:
- Run `import Pkg; Pkg.add("RealNeuralNetworks")` to install the RealNeuralNetworks package.


In [71]:
# Pearson correlation coefficient matrix
correlationMatrix = zero(distanceMatrix)
for i in 1:size(distanceMatrix, 1)
    for j in i:size(distanceMatrix, 2)
        c = cor( sampleMatrix[:,i], sampleMatrix[:,j] )
        correlationMatrix[i,j] = c
        correlationMatrix[j,i] = c
    end 
end

# rearrange distance matrix according to order
reorderedCorrelationMatrix = zero(correlationMatrix)
for (i,o1) in enumerate(clust.order)
    for (j, o2) in enumerate(clust.order)
        reorderedCorrelationMatrix[i,j] = correlationMatrix[o1,o2]
    end 
end 
PyPlot.imshow(reorderedCorrelationMatrix)
title("Pearson correlation coefficient matrix")

colorbar()

PyPlot.savefig(expanduser("~/Pictures/correlation_coefficient.png"))

LoadError: UndefVarError: distanceMatrix not defined

In [72]:
#fractalDimensionList = Vector{Float64}()
# @everywhere using RealNeuralNetworks.BranchNets
#addprocs(4)
#@time fractalDimensionList = map( n -> BranchNets.get_fractal_dimension(n)[1], values(branchNets) )